In [22]:
import math
import pandas as pd 
from numpy import *
#香农公式计算信息熵
def calcShannonEnt(dataSet):
    casenum = len(dataSet)
    caselabel = [data[-1] for data in dataSet] #从数据集中得到类别标签
    #得到类别为key，出现次数value的字典
    labelCounts = dict([(i,caselabel.count(i)) for i in caselabel])   
    infoEntropy = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/casenum
        infoEntropy -= prob*math.log(prob, 2)
    return infoEntropy
      
def CreateDataSet():
    f = open(r'D:\Adaboost等\iris_sample_224.csv','r', encoding='utf-8-sig')
    res = pd.read_csv(f)
    dataMat =  res.iloc[:,2:]
    m,n =shape(dataMat)
    dataSet = []
    for i in range(m):
        dataSet.append(list(dataMat.iloc[i,:].values))
    attrinames = list(res.columns[2:-1])  
    return dataSet,attrinames

#划分数据集：得到样本子集
def splitDataSet(dataSet, axis, value):
    subDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:#提取所有特征featVec[axis]的值为value的样本
            subFeatVec = featVec[:axis]
            subFeatVec.extend(featVec[axis+1:])
            subDataSet.append(subFeatVec)
    return subDataSet

def splitDataSet_c(dataSet, axis, value, LorR='L'):
    retDataSet = []
    featVec = []
    if LorR == 'L':
        for i,featVec in enumerate(dataSet):
            if float(featVec[axis]) < value:
                retDataSet.append(featVec)
                
    else:
        for i,featVec in enumerate(dataSet):
            if float(featVec[axis]) > value:
                retDataSet.append(featVec)
                 
                
    return retDataSet

def splitDataSet_caselabel(dataSet, axis, value, LorR='L'):
    retDataSet = []
    featVec = []
    if LorR == 'L':
        #print('left')
        for i,featVec in enumerate(dataSet):
            if float(featVec[axis]) < value:
                retDataSet.append(featVec)
                #print(str(i),end = ',')
    else:
        #print('\n'+'right')
        for i,featVec in enumerate(dataSet):
            if float(featVec[axis]) > value:
                retDataSet.append(featVec)
                #print(str(i),end = ',')
                
    return retDataSet
 
def chooseBestFeatureToSplit_c(dataSet, labelProperty):
    numFeatures = len(labelProperty)  # 特征数
    baseEntropy = calcShannonEnt(dataSet)  # 计算根节点的信息熵
    bestInfoGain = 0.0
    bestFeature = -1
    bestPartValue = None  # 连续的特征值，最佳划分值
    attrinames = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
    for i in range(numFeatures):  # 对每个特征循环
        print(attrinames[i])
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)  # 该特征包含的所有值
        newEntropy = 0.0
        bestPartValuei = None
        if labelProperty[i] == 0:  # 对离散的特征
            for value in uniqueVals:  # 对每个特征值，划分数据集, 计算各子集的信息熵
                subDataSet = splitDataSet(dataSet, i, value)
                prob = len(subDataSet) / float(len(dataSet))
                newEntropy += prob * calcShannonEnt(subDataSet)
        else:  # 对连续的特征
            sortedUniqueVals = list(uniqueVals)  # 对特征值排序
            sortedUniqueVals.sort()
            listPartition = []
            minEntropy = inf
            for j in range(len(sortedUniqueVals) - 1):  # 计算划分点
                partValue = round((float(sortedUniqueVals[j]) + float(
                    sortedUniqueVals[j + 1])) / 2,2)
                
                # 对每个划分点，计算信息熵
                dataSetLeft = splitDataSet_c(dataSet, i, partValue, 'L')
                dataSetRight = splitDataSet_c(dataSet, i, partValue, 'R')
                probLeft = len(dataSetLeft) / float(len(dataSet))
                probRight = len(dataSetRight) / float(len(dataSet))
                Entropy = probLeft * calcShannonEnt(
                    dataSetLeft) + probRight * calcShannonEnt(dataSetRight)
                if Entropy < minEntropy:  # 取最小的信息熵
                    minEntropy = Entropy
                    bestPartValuei = partValue
            newEntropy = minEntropy
            infoGaincon = baseEntropy-newEntropy
            print('bestpartvalue:',bestPartValuei)
            print('{2:.3f}={0:.3f}-{1:.3f}'.format(baseEntropy,newEntropy,infoGaincon))
        infoGain = baseEntropy - newEntropy  # 计算信息增益
        if infoGain > bestInfoGain:  # 取最大的信息增益对应的特征
            bestInfoGain = infoGain
            bestFeature = i
            bestPartValue = bestPartValuei
    return bestFeature, bestPartValue

# 创建树, 样本集 特征 特征属性（0 离散， 1 连续）
def createTree_c(dataSet, labels, labelProperty):
    # print dataSet, labels, labelProperty
    classList = [example[-1] for example in dataSet]  # 类别向量
    if classList.count(classList[0]) == len(classList):  # 如果只有一个类别，返回
        return classList[0]
    if len(dataSet[0]) == 1:  # 如果所有特征都被遍历完了，返回出现次数最多的类别
        return majorityCnt(classList)
    bestFeat, bestPartValue = chooseBestFeatureToSplit_c(dataSet,
                                                        labelProperty)  # 最优分类特征的索引
    print('\n'+'bestFeat:',labels[bestFeat])
    print('\n'+'bestPartValue:',bestPartValue)
    if bestFeat == -1:  # 如果无法选出最优分类特征，返回出现次数最多的类别
        return majorityCnt(classList)
    if labelProperty[bestFeat] == 0:  # 对离散的特征
        bestFeatLabel = labels[bestFeat]
        myTree = {bestFeatLabel: {}}
        labelsNew = copy.copy(labels)
        labelPropertyNew = copy.copy(labelProperty)
        del (labelsNew[bestFeat])  # 已经选择的特征不再参与分类
        del (labelPropertyNew[bestFeat])
        featValues = [example[bestFeat] for example in dataSet]
        uniqueValue = set(featValues)  # 该特征包含的所有值
        for value in uniqueValue:  # 对每个特征值，递归构建树
            subLabels = labelsNew[:]
            subLabelProperty = labelPropertyNew[:]
            subDataSet = splitDataSet(dataSet, bestFeat, value)
            if len(subDataset)== 0:
                myTree[bestFeatLabel][value]= majorityCnt(classList) 
            else:
                myTree[bestFeatLabel][value] = createTree_c(subDataSet, subLabels,subLabelProperty)
    else:  # 对连续的特征，不删除该特征，分别构建左子树和右子树
        bestFeatLabel = labels[bestFeat] + '<' + str(bestPartValue)
        myTree = {bestFeatLabel: {}}
        subLabels = labels[:]
        subLabelProperty = labelProperty[:]
        # 构建左子树
        valueLeft = '是'
        splitDataSet_caselabel(dataSet, bestFeat, bestPartValue, 'L')
        myTree[bestFeatLabel][valueLeft] = createTree_c(
            splitDataSet_c(dataSet, bestFeat, bestPartValue, 'L'), subLabels,
            subLabelProperty)
        # 构建右子树
        valueRight = '否'
        splitDataSet_caselabel(dataSet, bestFeat, bestPartValue, 'R')
        myTree[bestFeatLabel][valueRight] = createTree_c(
            splitDataSet_c(dataSet, bestFeat, bestPartValue, 'R'), subLabels,
            subLabelProperty)
    return myTree

labelProperty = [1,1,1,1]
myDat,attrinames = CreateDataSet()
print(attrinames)
tree = createTree_c(myDat,attrinames,labelProperty)#创建树
 
print (tree)  

['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
SepalLengthCm
bestpartvalue: 5.65
0.311=1.000-0.689
SepalWidthCm
bestpartvalue: 3.15
0.191=1.000-0.809
PetalLengthCm
bestpartvalue: 4.7
0.655=1.000-0.345
PetalWidthCm
bestpartvalue: 1.7
0.549=1.000-0.451

bestFeat: PetalLengthCm

bestPartValue: 4.7
SepalLengthCm
bestpartvalue: 7.0
0.054=0.592-0.537
SepalWidthCm
bestpartvalue: 3.15
0.076=0.592-0.516
PetalLengthCm
bestpartvalue: 5.15
0.198=0.592-0.394
PetalWidthCm
bestpartvalue: 1.7
0.245=0.592-0.347

bestFeat: PetalWidthCm

bestPartValue: 1.7
SepalLengthCm
bestpartvalue: 6.6
0.322=0.971-0.649
SepalWidthCm
bestpartvalue: 2.65
0.420=0.971-0.551
PetalLengthCm
bestpartvalue: 4.95
0.322=0.971-0.649
PetalWidthCm
bestpartvalue: 1.55
0.322=0.971-0.649

bestFeat: SepalWidthCm

bestPartValue: 2.65
SepalLengthCm
bestpartvalue: 6.15
0.252=0.918-0.667
SepalWidthCm
bestpartvalue: 2.75
0.252=0.918-0.667
PetalLengthCm
bestpartvalue: 5.0
0.252=0.918-0.667
PetalWidthCm
bestpartvalue: 1.55